**Extracting the stroke dataset from S3 bucket**

In [2]:
import os
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()
import pandas as pd


Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-05-05 15:07:04--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-05-05 15:07:04 (5.97 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url ="https://project-4.s3.eu-west-2.amazonaws.com/healthcare-dataset-stroke-data.csv"
spark.sparkContext.addFile(url)
stroke_data = spark.read.csv(SparkFiles.get("healthcare-dataset-stroke-data.csv"), header=True, inferSchema=True)

stroke_data.show(10)

+---+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----------+---------------+------+
|_c0|   id|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level|        bmi| smoking_status|stroke|
+---+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----------+---------------+------+
|  0| 9046|  Male|67.0|           0|            1|         Yes|      Private|         Urban|           228.69|       36.6|formerly smoked|     1|
|  1|51676|Female|61.0|           0|            0|         Yes|Self-employed|         Rural|           202.21|28.89323691|   never smoked|     1|
|  2|31112|  Male|80.0|           0|            1|         Yes|      Private|         Rural|           105.92|       32.5|   never smoked|     1|
|  3|60182|Female|49.0|           0|            0|         Yes|      Private|         Urban|           171.23|       34.4|  

In [11]:
stroke_df = stroke_data.select(["id","gender", "age", "hypertension", "heart_disease", "ever_married","work_type","Residence_type","avg_glucose_level","bmi","smoking_status","stroke"])
stroke_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



 **Load the DataFrame**

In [14]:
mode="append"
jdbc_url = "jdbc:postgresql://<ENDPOINT REMOVED>/stroke_data"
config = {"user":"root",
          "password": "<PASSWORD REMOVED>",
          "driver":"org.postgresql.Driver"}

In [15]:
#Write dataframe to stroke_dataset
stroke_df.write.jdbc(url=jdbc_url, table='stroke_dataset', mode=mode, properties=config)